In [1]:
import os
import json
import requests

- *_get_location_url_* is the route in the api
- *_upload_key_* is the name of the artifact
- *_folder_* is the folder you would like to store it in

In [2]:
get_location_url = "https://la7nxehzwg.execute-api.af-south-1.amazonaws.com/alpha/live/uploadclip/"
upload_key = "test_prefilled_post.txt"
folder = "video"

*Firstly*, you get the presigned link.
- This returns all the fields you require to upload to the S3 bucket


In [3]:
resp = requests.post(get_location_url, params={"upload_key": upload_key, "folder": folder})
print(resp)

<Response [200]>


In [4]:
response_dict = json.loads(resp.text)

In [5]:
with open(upload_key, 'rb') as binary_object:
    files = {'file': (upload_key, binary_object)}
    http_response = requests.post(response_dict['url'], data=response_dict['fields'], files=files)